In [118]:
import scrapy
from scrapy.http import TextResponse, HtmlResponse
import requests

In [4]:
# Create the spider class
class YourSpider( scrapy.Spider ):
    name = "your_spider"
    
    # start_requests method
    def start_requests( self ):
        urls = ['https://www.datacamp.com', "https://scrapy.org"]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
            
    # parse method
    def parse( self, response ):
        pass


In [126]:
# Spiders to get autors:

class DCspider( scrapy.Spider ):
    name = 'dcspider'
  # start_requests method
    def start_requests( self ):
        
        url = 'https://assets.datacamp.com/production/repositories/2560/datasets/19a0a26daa8d9db1d920b5d5607c19d6d8094b3b/all_short' 
        resp = requests.get(url)
        resp = TextResponse(body=resp.content, url=url)
        
        author_names = resp.css('.course-block__author-name::text').extract()
        
    # Here we will just return the list of Authors
        return author_names

In [129]:
foo = DCspider()
foo.start_requests()

['Jonathan Cornelissen',
 'Matt Dowle',
 'Garrett Grolemund',
 'Garrett Grolemund',
 'Garrett Grolemund',
 'Filip Schouwenaars',
 'Gilles Inghelbrecht',
 'Nick Carchedi',
 'Filip Schouwenaars',
 'Filip Schouwenaars',
 'Mark Peterson']

In [130]:
def main():
    foo = DCspider()
    lista = foo.start_requests()
    for i in lista:
        print(i)

if __name__ == "__main__":
    main()

Jonathan Cornelissen
Matt Dowle
Garrett Grolemund
Garrett Grolemund
Garrett Grolemund
Filip Schouwenaars
Gilles Inghelbrecht
Nick Carchedi
Filip Schouwenaars
Filip Schouwenaars
Mark Peterson


In [146]:

# Create the Spider class
class DCdescr2( scrapy.Spider ):
    name = 'dcdescr'
  
   # start_requests method
    def start_requests( self ):
        yield scrapy.Request( url = url_short, callback = self.parse )
  
    # First parse method
    def parse( self, response ):
        links = response.css( 'div.course-block > a::attr(href)' ).extract()
    # Follow each of the extracted links
        for link in links:
              yield response.follow( url = link, callback = self.parse_descr )
      
  # Second parsing method
    def parse_descr( self, response ):
        # Extract course description
        course_descr = response.css( 'p.course__description::text' ).extract_first()
        # For now, just yield the course description
        yield course_descr


        
def inspect_spider( s ):
    news = s()
    try:
        req1 = list( news.start_requests() )[0]
        html1 = requests.get( req1.url ).content
        response1 = TextResponse( url = req1.url, body = html1, encoding = 'utf-8' )
        req2 = list( news.parse( response1 ) )[0]
        html2 = requests.get( req2.url ).content
        response2 = TextResponse( url = req2.url, body = html2, encoding = 'utf-8' )
        for d in news.parse_descr( response2 ):
            print("One course description you found is:", d )
            break
    except:
        print("Oh no! Something is wrong with the code. Keep trying!")
        
url_short = 'https://assets.datacamp.com/production/repositories/2560/datasets/19a0a26daa8d9db1d920b5d5607c19d6d8094b3b/all_short'

# Inspect the spider
inspect_spider( DCdescr2 )

One course description you found is: In this introduction to R, you will master the basics of this beautiful open source language, including factors, lists and data frames. With the knowledge gained in this course, you will be ready to undertake your first very own data analysis. With over 2 million users worldwide R is rapidly becoming the leading programming language in statistics and data science. Every year, the number of R users grows by 40% and an increasing number of organizations are using it in their day-to-day activities. Leverage the power of R by completing this free R online course today!


In [147]:
# Spyder with 2 leves 

# Import the CrawlerProcess: for running the spider
from scrapy.crawler import CrawlerProcess

# Create the Spider class
class DC_Chapter_Spider(scrapy.Spider):
    name = "dc_chapter_spider"
    
    # start_requests method
    def start_requests(self):
        yield scrapy.Request(url = url_short,
                             callback = self.parse_front)
    # First parsing method
    def parse_front(self, response):
        course_blocks = response.css('div.course-block')
        course_links = course_blocks.xpath('./a/@href')
        links_to_follow = course_links.extract()
        for url in links_to_follow:
            yield response.follow(url = url,
                                  callback = self.parse_pages)
  
    # Second parsing method
    def parse_pages(self, response):
        crs_title = response.xpath('//h1[contains(@class,"title")]/text()')
        crs_title_ext = crs_title.extract_first().strip()
        ch_titles = response.css('h4.chapter__title::text')
        ch_titles_ext = [t.strip() for t in ch_titles.extract()]
        dc_dict[ crs_title_ext ] = ch_titles_ext



In [148]:
# Initialize the dictionary **outside** of the Spider class
dc_dict = dict()

url_short = 'https://assets.datacamp.com/production/repositories/2560/datasets/19a0a26daa8d9db1d920b5d5607c19d6d8094b3b/all_short'


# Run the Spider
process = CrawlerProcess()
process.crawl(DC_Chapter_Spider)
process.start()

2019-10-14 15:55:05 [scrapy.utils.log] INFO: Scrapy 1.7.3 started (bot: scrapybot)
2019-10-14 15:55:05 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.7.0, Python 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.6.1, Platform Windows-10-10.0.17763-SP0
2019-10-14 15:55:05 [scrapy.crawler] INFO: Overridden settings: {}
2019-10-14 15:55:05 [scrapy.extensions.telnet] INFO: Telnet Password: 67839894d38fa54b
2019-10-14 15:55:05 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2019-10-14 15:55:06 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defa

In [155]:
# Print a preview of courses
for i in dc_dict:
    print(i, '\n')
    lista = dc_dict[i]
    for j in lista:
        print(j)
    print('\n \n')

Data Manipulation in R with dplyr 

Introduction to dplyr and tbls
Filter and arrange
Group_by and working with databases
Select and mutate
Summarize and the pipe operator
Introduction to dplyr and tbls
Select and mutate
Filter and arrange
Summarize and the pipe operator
Group_by and working with databases

 

Reporting with R Markdown 

Authoring R Markdown Reports
Embedding Code
Compiling Reports
Configuring R Markdown (optional)

 

Intermediate R - Practice 

Conditionals and Control Flow
Loops
Functions
The apply family
Utilities

 

Data Visualization in R with ggvis 

The Grammar of Graphics
Transformations
Customizing Axes, Legends, and Scales
Lines and Syntax
Interactivity and Layers
The Grammar of Graphics
Lines and Syntax
Transformations
Interactivity and Layers
Customizing Axes, Legends, and Scales

 

Intermediate R 

Conditionals and Control Flow
Functions
Utilities
Loops
The apply family
Conditionals and Control Flow
Loops
Functions
The apply family
Utilities

 

Cleanin

In [159]:
# Import scrapy
import scrapy

# Import the CrawlerProcess: for running the spider
from scrapy.crawler import CrawlerProcess

url_short = 'https://assets.datacamp.com/production/repositories/2560/datasets/19a0a26daa8d9db1d920b5d5607c19d6d8094b3b/all_short'


# Create the Spider class
class DC_Description_Spider(scrapy.Spider):
    name = "dc_chapter_spider"
  # start_requests method
    def start_requests(self):
        yield scrapy.Request(url = url_short,
                         callback = self.parse_front)
  # First parsing method
    def parse_front(self, response):
        course_blocks = response.css('div.course-block')
        course_links = course_blocks.xpath('./a/@href')
        links_to_follow = course_links.extract()
        for url in links_to_follow:
            yield response.follow(url = url,
                            callback = self.parse_pages)
  
    # Second parsing method
    def parse_pages(self, response):
    # Create a SelectorList of the course titles text
        crs_title = response.xpath('//h1[contains(@class,"title")]/text()')
    # Extract the text and strip it clean
        crs_title_ext = crs_title.extract_first().strip()
    # Create a SelectorList of course descriptions text
        crs_descr = response.css( 'p.course__description::text' )
    # Extract the text and strip it clean
        crs_descr_ext = crs_descr.extract_first().strip()
    # Fill in the dictionary
        dc_dict[crs_title_ext] = crs_descr_ext



In [160]:
# Initialize the dictionary **outside** of the Spider class
dc_dict = dict()

# Run the Spider
process = CrawlerProcess()
process.crawl(DC_Description_Spider)
process.start()

dc_dict

2019-10-14 16:49:33 [scrapy.utils.log] INFO: Scrapy 1.7.3 started (bot: scrapybot)
2019-10-14 16:49:33 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.7.0, Python 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.6.1, Platform Windows-10-10.0.17763-SP0
2019-10-14 16:49:33 [scrapy.crawler] INFO: Overridden settings: {}
2019-10-14 16:49:33 [scrapy.extensions.telnet] INFO: Telnet Password: 820e890f33c2d627
2019-10-14 16:49:33 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2019-10-14 16:49:33 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defa

ReactorNotRestartable: 